Importing the dataset

In [ ]:
import pandas as pd
dataset = pd.read_csv("/content/drive/MyDrive/CGM/data _1.csv")
print(dataset)

     Intensity  Wavelength  Spectrum  ... Unnamed: 11  Unnamed: 12 Unnamed: 13
0           15    926.2256       H I  ...         NaN          NaN         NaN
1           20    930.7482       H I  ...         NaN          NaN         NaN
2           30    937.8034       H I  ...         NaN          NaN         NaN
3           50    949.7430       H I  ...         NaN          NaN         NaN
4          100    972.5367       H I  ...         NaN          NaN         NaN
..         ...         ...       ...  ...         ...          ...         ...
257        150  10691.2500       C I  ...         NaN          NaN         NaN
258         90  11748.2200       C I  ...         NaN          NaN         NaN
259        140  11753.3200       C I  ...         NaN          NaN         NaN
260        110  11754.7600       C I  ...         NaN          NaN         NaN
261         50  16890.3800       C I  ...         NaN          NaN         NaN

[262 rows x 14 columns]


In [ ]:
import matplotlib as plt
import numpy as np

In [ ]:
X = dataset.iloc[:, 0:2].values
Y = dataset.iloc[:, 2:3].values

In [ ]:
print(Y)

Label Encoding the dependent variable

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y = le.fit_transform(Y)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
print(Y)

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 5 5 5 5 5 5 6 6 5 6 6 6 6 6 6 6 6 5 5 5 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 4 4 4 3 4 4 4 4 4 4 4 4 4 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4
 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 3 4 3 3 3 3 3 3 3 3 4 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 0 0 0 0 0 0
 0 0 0]


Splitting the dataset into Training set and Test Set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 0)

Hyperparameter tuning XGBoost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import precision_score,recall_score,accuracy_score,f1_score,roc_auc_score
from scipy import stats
from scipy.stats import randint

clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic')
param_dist = {'n_estimators': stats.randint(150, 1000),
              'learning_rate': stats.uniform(0.01, 0.6),
              'subsample': stats.uniform(0.3, 0.9),
              'max_depth': [3, 4, 5, 6, 7, 8, 9],
              'colsample_bytree': stats.uniform(0.5, 0.9),
              'min_child_weight': [1, 2, 3, 4]
             }

clf = RandomizedSearchCV(clf_xgb, 
                         param_distributions = param_dist,
                         cv = 2,  
                         n_iter = 100, 
                         scoring = 'accuracy', 
                         error_score = 0, 
                         verbose = 3, 
                         n_jobs = -1)
clf.fit(X_train,Y_train)

Fitting 2 folds for each of 100 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    9.0s finished


RandomizedSearchCV(cv=2, error_score=0,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=1, nthread=None,
                                           objective='binary:logistic',
                                           random_state=0, reg_alpha=0,
                                           reg_lambda=1, scal...
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe523e05710>,
                                        'max_dep

In [ ]:
clf.best_params_

{'colsample_bytree': 0.805363959790586,
 'learning_rate': 0.289442377680034,
 'max_depth': 8,
 'min_child_weight': 1,
 'n_estimators': 514,
 'subsample': 0.8175746415373166}

XGBoost Classifier

In [ ]:
from xgboost import XGBClassifier
classifier = XGBClassifier(colsample_bytree=0.805363959790586,learning_rate=0.289442377680034,max_depth=8,min_child_weight=1,n_estimators=514,subsample= 0.8175746415373166)
classifier.fit(X_train, Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.805363959790586, gamma=0,
              learning_rate=0.289442377680034, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=None, n_estimators=514, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8175746415373166, verbosity=1)

Confusion Matrix to determine the accuracy

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
Y_pred = classifier.predict(X_test)
cm = confusion_matrix(Y_test, Y_pred) 
print(cm)
accuracy_score(Y_test, Y_pred)

[[ 6  0  0  0  0  1  0]
 [ 0  6  0  0  0  0  0]
 [ 1  1  4  0  1  1  0]
 [ 1  1  0 13  0  0  0]
 [ 0  0  2  0  7  0  2]
 [ 0  0  0  0  0  9  0]
 [ 0  0  0  0  0  0 10]]


0.8333333333333334

In [ ]:
print(Y_pred)

[3 4 4 2 5 1 1 3 0 3 1 5 5 4 3 4 6 6 3 5 6 0 0 0 0 6 6 1 1 6 5 6 0 3 1 0 3
 2 6 6 3 1 5 2 4 4 5 0 6 3 2 5 3 6 5 6 3 1 4 4 5 3 2 3 5 2]


In [ ]:
print(Y_test)

[3 4 4 2 5 1 1 3 3 3 3 5 0 4 3 4 6 6 3 5 6 0 2 0 0 4 6 1 1 6 5 6 0 3 1 0 3
 2 6 4 3 2 2 4 4 2 5 0 6 3 4 5 3 6 5 6 3 1 4 4 5 3 2 3 5 2]


Predicting a single prediction

In [ ]:
le.inverse_transform(classifier.predict([[250,1340]]))

array(['C II'], dtype=object)